In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
import geopy.distance
from sklearn.ensemble import RandomForestClassifier
import pickle
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder

# Initialize Firebase Admin SDK
def initialize_firebase():
    try:
        if not firebase_admin._apps:
            cred = credentials.Certificate('serviceAccountKey.json')
            firebase_admin.initialize_app(cred)
        else:
            print("Firebase already initialized.")
    except Exception as e:
        print(f"Error initializing Firebase: {e}")

initialize_firebase()

# Initialize Firestore client
db = firestore.client()

In [3]:
# Fetch employee and job site data
employees_ref = db.collection('employees')
job_sites_ref = db.collection('job_sites')

employees = [doc.to_dict() for doc in employees_ref.stream()]
job_sites = [doc.to_dict() for doc in job_sites_ref.stream()]

In [4]:
# Geocoding function
geolocator = Nominatim(user_agent="OptiShiftApp")

def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for address: {address}")
            return None, None
    except Exception as e:
        print(f"Geocoding error for address {address}: {e}")
        return None, None

# Update employee and job site data with latitude and longitude
for employee in employees:
    lat, lon = geocode_address(employee['home_address'])
    if lat is not None and lon is not None:
        employee['latitude'] = lat
        employee['longitude'] = lon
    else:
        print(f"Skipping employee {employee.get('worker_id')} due to missing coordinates.")

for site in job_sites:
    lat, lon = geocode_address(site['location'])
    if lat is not None and lon is not None:
        site['latitude'] = lat
        site['longitude'] = lon
    else:
        print(f"Skipping job site {site.get('site_id')} due to missing coordinates.")

In [8]:
# Function to calculate distance between two sets of coordinates
def calculate_distance(coord1, coord2):
    if coord1 and coord2:
        return geopy.distance.geodesic(coord1, coord2).km
    else:
        return float('inf')  # Return infinity if coordinates are missing

# Function to match employees to job sites
def match_employees_to_job_sites(employees, job_sites):
    assignments = {}
    for site in job_sites:
        site_id = site["site_id"]
        site_coords = (site.get("latitude"), site.get("longitude"))
        if None in site_coords:
            print(f"Skipping job site {site_id} due to missing coordinates.")
            continue

        # Filter employees based on role and availability
        filtered_employees = []
        for employee in employees:
            employee_coords = (employee.get("latitude"), employee.get("longitude"))
            if None in employee_coords:
                print(f"Skipping employee {employee.get('worker_id')} due to missing coordinates.")
                continue

            # Check role match
            if employee["role"] not in site["required_roles"]:
                continue

            # Check availability match
            if not any(schedule in employee["availability"] for schedule in site["work_schedule"]):
                continue

            # Calculate distance
            distance = calculate_distance(employee_coords, site_coords)

            # Add employee to filtered list
            filtered_employees.append({
                "worker_id": employee["worker_id"],
                "name": f"{employee['first_name']} {employee['sur_name']}",
                "rating": employee["rating"],
                "distance": distance
            })

        # If there are matching employees, assign the closest one
        if filtered_employees:
            # Sort employees by distance (closest first)
            filtered_employees.sort(key=lambda x: x["distance"])
            closest_employee = filtered_employees[0]
            assignments[site_id] = closest_employee
        else:
            print(f"No matching employees found for job site {site_id}.")
            assignments[site_id] = None

    return assignments

# Example usage
assignments = match_employees_to_job_sites(employees, job_sites)
for site_id, employee in assignments.items():
    if employee:
        print(f"Job Site: {site_id}")
        print(f"  - Assigned Employee: {employee['name']} (Rating: {employee['rating']}, Distance: {employee['distance']:.2f} km)")
    else:
        print(f"Job Site: {site_id} - No employee assigned.")

Skipping employee K3CLRFW1 due to missing coordinates.
Skipping employee YKEAMHNI due to missing coordinates.
No matching employees found for job site SITE1256.
Skipping employee K3CLRFW1 due to missing coordinates.
Skipping employee YKEAMHNI due to missing coordinates.
No matching employees found for job site SITE1877.
Skipping employee K3CLRFW1 due to missing coordinates.
Skipping employee YKEAMHNI due to missing coordinates.
No matching employees found for job site SITE1033.
Skipping employee K3CLRFW1 due to missing coordinates.
Skipping employee YKEAMHNI due to missing coordinates.
No matching employees found for job site SITE6928.
Skipping employee K3CLRFW1 due to missing coordinates.
Skipping employee YKEAMHNI due to missing coordinates.
No matching employees found for job site SITE9352.
Skipping employee K3CLRFW1 due to missing coordinates.
Skipping employee YKEAMHNI due to missing coordinates.
No matching employees found for job site SITE6630.
Skipping employee K3CLRFW1 due to 